# L'ajout des noeuds dans la base de donnees neo4j

In [1]:
import json
import random
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))
collaborator_news = random.randint(20, 90)
manager_news = random.randint(20, 90)


def add_entire_structure_with_manager(tx, entreprise_name, direction_manager, service_manager, equipe_manager, direction_name, service_name, equipe_name, first_name, last_name, age, motivation, email):
    # This Cypher query creates nodes for all entities and relationships between them
    query = """
    MERGE (entreprise:Entreprise {nom: $entreprise_name})
    MERGE (entreprise)-[:HAS_DIRECTION]->(direction:Structure {nom: $direction_name, type: 'direction'})
    MERGE (direction)-[:HAS_SERVICE]->(service:Structure {nom: $service_name, type: 'service'})
    MERGE (service)-[:HAS_EQUIPE]->(equipe:Structure {nom: $equipe_name, type: 'equipe'})
    MERGE (equipe)-[:HAS_MANAGER]->(equipe_manager:Collaborateur {prenom: $equipe_manager_prenom, nom: $equipe_manager_nom, email: $equipe_manager_email, Motivation: $equipe_manager_motivation})
    MERGE (direction)-[:HAS_MANAGER]->(direction_manager:Collaborateur {prenom: $direction_manager_prenom, nom: $direction_manager_nom, email: $direction_manager_email, motivation: $direction_manager_motivation})
    MERGE (service)-[:HAS_MANAGER]->(service_manager:Collaborateur {prenom: $service_manager_prenom, nom: $service_manager_nom, email: $service_manager_email, motivation: $service_manager_motivation})
    MERGE (employee:Collaborateur {prenom: $first_name, nom: $last_name, age: $age, email: $email, motivation: $employee_motivation})
    MERGE (equipe)-[:HAS_EMPLOYEE]->(employee)
    """
    tx.run(query, entreprise_name=entreprise_name, direction_manager_prenom=direction_manager['prenom'], direction_manager_nom=direction_manager['nom'], direction_manager_email=direction_manager['email'], direction_manager_motivation=direction_manager['motivation'], service_manager_prenom=service_manager['prenom'], service_manager_nom=service_manager['nom'], service_manager_email=service_manager['email'], service_manager_motivation=service_manager['motivation'], equipe_manager_prenom=equipe_manager['prenom'], equipe_manager_nom=equipe_manager['nom'], equipe_manager_email=equipe_manager['email'], equipe_manager_motivation=equipe_manager['motivation'], direction_name=direction_name, service_name=service_name, equipe_name=equipe_name, first_name=first_name, last_name=last_name, age=age, email=email, employee_motivation=motivation)

def add_manager_par_relation(tx, direction_manager_nom, direction_manager_prenom, manager_par_nom, manager_par_prenom):
# This Cypher query creates a "MANAGER_PAR" relationship between two managers
    query = """
    MATCH (m1:Manager {nom: $direction_manager_nom, prenom: $direction_manager_prenom})
    MATCH (m2:Manager {nom: $manager_par_nom, prenom: $manager_par_prenom})
    MERGE (m1)-[:MANAGER_PAR]->(m2)
"""
    tx.run(query, direction_manager_nom=direction_manager_nom, direction_manager_prenom=direction_manager_prenom, manager_par_nom=manager_par_nom, manager_par_prenom=manager_par_prenom)

with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/nom_prenom.json") as fichier_test1:
    data1 = json.load(fichier_test1)

with open('C:/Users/ESPRI/Desktop/PFE/Neo4j/Entreprise.json') as f:
    data = json.load(f)
for entreprise in data:
    entreprise_name = entreprise['nom']
    used_names = set()  # Set to keep track of used names
    used_emails = set()  # Set to keep track of used emails

    for direction in entreprise['children']:
        direction_name = direction['nom']
        
        # Randomly choose a manager for the direction
        aleatoire_direction_manager_prenom = random.choice(data1['prenom'])
        aleatoire_direction_manager_nom = random.choice(data1['nom'])
        aleatoire_direction_manager_email = aleatoire_direction_manager_prenom.lower() + '.' + aleatoire_direction_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

        direction_manager = {'prenom': aleatoire_direction_manager_prenom, 'nom': aleatoire_direction_manager_nom, 'email': aleatoire_direction_manager_email, 'motivation': manager_news}

        # Add the direction manager to the used names and emails sets
        used_names.add((direction_manager['prenom'], direction_manager['nom']))
        used_emails.add(direction_manager['email'])

        for service in direction['children']:
            service_name = service['nom']

            # Randomly choose a manager for the service
            aleatoire_service_manager_prenom = random.choice(data1['prenom'])
            aleatoire_service_manager_nom = random.choice(data1['nom'])
            aleatoire_service_manager_email = aleatoire_service_manager_prenom.lower() + '.' + aleatoire_service_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

            service_manager = {'prenom': aleatoire_service_manager_prenom, 'nom': aleatoire_service_manager_nom, 'email': aleatoire_service_manager_email, 'motivation': manager_news}

            # Add the service manager to the used names and emails sets
            used_names.add((service_manager['prenom'], service_manager['nom']))
            used_emails.add(service_manager['email'])

            for equipe in service['children']:
                equipe_name = equipe['nom']

                # Randomly choose a manager for the team
                aleatoire_equipe_manager_prenom = random.choice(data1['prenom'])
                aleatoire_equipe_manager_nom = random.choice(data1['nom'])
                aleatoire_equipe_manager_email = aleatoire_equipe_manager_prenom.lower() + '.' + aleatoire_equipe_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                equipe_manager = {'prenom': aleatoire_equipe_manager_prenom, 'nom': aleatoire_equipe_manager_nom, 'email': aleatoire_equipe_manager_email, 'motivation': manager_news}

                # Add the team manager to the used names and emails sets
                used_names.add((equipe_manager['prenom'], equipe_manager['nom']))
                used_emails.add(equipe_manager['email'])

                for i in range(5):
                    # Randomly choose a collaborator and generate a unique email
                    aleatoire_collaborateur_prenom = random.choice(data1['prenom'])
                    aleatoire_collaborateur_nom = random.choice(data1['nom'])
                    aleatoire_collaborateur_email = aleatoire_collaborateur_prenom.lower() + '.' + aleatoire_collaborateur_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                    # Check if the generated email is not already used
                    while aleatoire_collaborateur_email in used_emails:
                        aleatoire_collaborateur_prenom = random.choice(data1['prenom'])
                        aleatoire_collaborateur_nom = random.choice(data1['nom'])
                        aleatoire_collaborateur_email = aleatoire_collaborateur_prenom.lower() + '.' + aleatoire_collaborateur_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                    # Add the collaborator email to the used emails set
                    used_emails.add(aleatoire_collaborateur_email)

                    # Randomly generate the number of news for the collaborator
                    aleatoire_collaborateur_motivation = random.randint(0, collaborator_news)

                    # Add the collaborator to the graph
                    with driver.session() as session:
                        session.execute_write(add_entire_structure_with_manager, entreprise_name, direction_manager, service_manager, equipe_manager, direction_name, service_name, equipe_name, aleatoire_collaborateur_prenom, aleatoire_collaborateur_nom, 30, aleatoire_collaborateur_motivation, aleatoire_collaborateur_email)

                # Create "MANAGER_PAR" relationships between managers
                with driver.session() as session:
                    session.execute_write(add_manager_par_relation, direction_manager['nom'], direction_manager['prenom'], service_manager['nom'], service_manager['prenom'])
                    session.execute_write(add_manager_par_relation, service_manager['nom'], service_manager['prenom'], equipe_manager['nom'], equipe_manager['prenom'])

# Ajout de la date de début contrat et date de départ

In [2]:
from neo4j import GraphDatabase
import random
import json
from datetime import datetime, timedelta

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Lecture du fichier JSON contenant les dates
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/Date_collaborateurs.json") as file:
    dates_data = json.load(file)

# Extraction des dates de début et de fin du contrat du fichier JSON
date_debut = datetime.strptime(dates_data["date_debut"], "%Y-%m-%d")
date_fin = datetime.strptime(dates_data["date_fin"], "%Y-%m-%d")

# Fonction pour générer une date aléatoire entre la date de début et la date de fin
def generate_random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    random_date = start + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

# Fonction pour ajouter les attributs à chaque collaborateur
def add_attributes_to_collaborateurs():
    with driver.session() as session:
        query = "MATCH (c:Collaborateur) RETURN c"
        result = session.run(query)

        # Parcourir chaque collaborateur
        for record in result:
            collaborateur = record["c"]

            # Vérifier si le collaborateur a une date de départ
            if "date_depart" in collaborateur:
                # Supprimer le collaborateur de la base de données
                delete_query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                DELETE c
                """
                session.run(delete_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"])
                continue  # Passer au collaborateur suivant

            # Générer une date de début de contrat aléatoire
            start_date = generate_random_date(date_debut, date_fin)

            # Ajouter l'attribut "date_debut_contrat" au collaborateur
            update_query = """
            MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
            SET c.date_debut_contrat = $start_date
            """
            session.run(update_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], start_date=start_date)

            # Générer une probabilité de départ
            if random.random() <= 0.1:
                # Le collaborateur a quitté l'entreprise, générer une date de départ aléatoire
                departure_date = generate_random_date(date_debut, date_fin)

                # Ajouter l'attribut "date_depart" au collaborateur
                update_query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                SET c.date_depart = $departure_date
                """
                session.run(update_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], departure_date=departure_date)

# Appel de la fonction pour ajouter les attributs aux collaborateurs
add_attributes_to_collaborateurs()

# Fermeture de la connexion à la base de données Neo4j
driver.close()


# Le nombre de publications chaque heure dans un tableau

In [3]:
#
import json
from tabulate import tabulate
import random
import math
from datetime import datetime

# Ouvrir le fichier JSON des mois
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/Moyen_mois.json") as f:
    data = json.load(f)

# Ouvrir le fichier JSON des heures
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/heures.json") as f:
    data1 = json.load(f)
    
data1["Heures"]=list(range(24))
    
# Convertir les données en liste de listes pour le tabulateur
print("Tableau initial des mois avec leurs moyennes")
table_mois = []
for i in range(len(data["Mois"])):
    table_mois.append([data["Mois"][i], data["Moyen"][i]])

    
# Afficher le tableau avec tabulate
print(tabulate(table_mois, headers=["Mois", "Moyenne"]))
    
# print("Le tableau  ")
# T=[]
# moyenne_du_jour = []
# for i in range(len(data["Mois"])):
#    moyenne = data["Moyen"][i]
#    moyenne_du_jour = moyenne + random.randint(-2, 2)
#    mois_new=data["Mois"][i]
#    T.append([mois_new,moyenne_du_jour])
# Afficher le résultat
# print(tabulate(T, headers=["un jour du mois", "Moyenne"]))



table_initiale_heures = []

for i in range(len(data1["Heures"])):
    table_initiale_heures.append([data1["Heures"][i], data1["Pourcentage"][i]])
print(tabulate(table_initiale_heures, headers=["Heures", "Pourcentage"]))

# Créer une copie du tableau initial
table_copy_heures = table_initiale_heures.copy()

# Trier aléatoirement la copie du tableau
random.shuffle(table_copy_heures)

# Afficher la copie triée aléatoirement avec tabulate
print(tabulate(table_copy_heures, headers=["Heures", "Pourcentage"]))



def rand_round (num):
    a = random.randint(0, 1)
    if(a==1):
        return math.ceil(num)
    else :
        return math.floor(num)
    
def get_month_from_date(date):
    month = date.month
    return month

# le nb de publications chaque heure dans un tableau
def plublications_par_heure(date):
    
    #nb de publications par heure
    resultats = []
    
    i = get_month_from_date(date) - 1
    mois = table_mois[i][0]
    moyenne_mois = table_mois[i][1]
    #print("moyenne du mois", moyenne_mois)
    rand_variation = random.randint(-2, 2)
    #print("variation aléatoire", rand_variation)
    nb_pub = rand_round( moyenne_mois + rand_variation)
    print("nombre de publications pour cette journée: ",nb_pub)
    reste = nb_pub
    for j in range(len(table_copy_heures)):
        heure = table_copy_heures[j][0]
        pourcentage = table_copy_heures[j][1]
        resultat = rand_round((nb_pub * pourcentage / 100))  # Calculer le résultat arrondi à l'entier le plus proche
        if resultat <= reste :
            resultats.append([heure, resultat])
            reste = reste - resultat
        else :
            resultats.append([heure, reste])
            reste = 0
    if reste != 0:
        resultats[len(resultats)-1][1] += reste

    sortedresults = sorted(resultats, key=lambda x: x[0])
    
    finalresults = [x[1] for x in sortedresults]
    
    # Afficher le tableau final results
    print(finalresults)
    
    return finalresults


plublications_par_heure(datetime(2023, 6, 18))


Tableau initial des mois avec leurs moyennes
Mois         Moyenne
---------  ---------
Janvier        20
fevrier        19.28
mars           17.45
avril          18.1
mai            19.5
juin           13.5
juillet        14.8
aout           12.6
septembre      14
october        15
novembre       18
decembre       16
  Heures    Pourcentage
--------  -------------
       0           0.1
       1           0.05
       2           0.05
       3           0.05
       4           0.05
       5           0.05
       6           0.05
       7           0.1
       8           3.1
       9           9.9
      10          10.85
      11          13.65
      12          11.85
      13           7.65
      14           7
      15           6.8
      16           8.95
      17           9.2
      18           4.1
      19           2.8
      20           1.9
      21           0.9
      22           0.5
      23           0.3
  Heures    Pourcentage
--------  -------------
       0           0.1
 

[1, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]

# Ajout des publications 

In [4]:
import json
import datetime
import random
from neo4j import GraphDatabase
from datetime import datetime
from datetime import timedelta
import random
from datetime import time
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase, basic_auth

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

query = """MATCH (c:Collaborateur)
WITH c, RAND() AS r
ORDER BY r
LIMIT 1
RETURN c"""


def get_collab():
    with driver.session() as session:
        result = session.run(query)
        collab = result.single()["c"]
        return collab


# Charger les dates de début et de fin à partir du fichier JSON
with open("c:/Users/ESPRI/Desktop/PFE/Neo4j/date.json", "r") as file:
    dates_data = json.load(file)
date_debut_str = dates_data["date_debut"]
date_fin_str = dates_data["date_fin"]

# Convertir les dates de chaînes de caractères en objets datetime
date_debut = datetime.strptime(date_debut_str, "%Y-%m-%d")
date_fin = datetime.strptime(date_fin_str, "%Y-%m-%d")


def iterate_datetime(start_datetime, end_datetime):
    current_datetime = start_datetime
    onehour = timedelta(hours=1)

    while current_datetime <= end_datetime:
        pub_per_h = plublications_par_heure(current_datetime)
        for h in range(24):
            hour_range = f"{h:02d}:00-{h:02d}:59"
            current_hour = current_datetime.hour
            print(current_datetime.date(), hour_range, pub_per_h[current_hour])

            random_value = random.randint(1, 100)
            nb = pub_per_h[current_hour]
            for i in range(nb):
                random_minute = random.randint(0, 59)
                random_hour = current_hour
                random_time = current_datetime.replace(hour=random_hour, minute=random_minute)

                if pub_per_h[current_hour] > 0:
                    collaborateur = get_collab()
                    random_value = random.randint(1, 100)
                    while collaborateur["motivation"] is None or random_value >= collaborateur["motivation"]:
                        collaborateur = get_collab()
                        random_value = random.randint(1, 100)
                    
                    date_debut_contrat = datetime.strptime(collaborateur["date_debut_contrat"], "%Y-%m-%d").date()
                    
                    if random_time.date() > date_debut_contrat:
                        print(f"{collaborateur['prenom']} {collaborateur['nom']} a créé une publication")
                        
                        with driver.session() as session:
                            query = '''
                            MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab}) 
                            CREATE (p:Publication {heure: $heure, date_creation: $date_creation})-[:IS_CREATED_BY]->(c)
                            '''
                            session.run(query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"],
                                        heure=str(random_time), date_creation=str(current_datetime.date()))

            current_datetime += onehour

iterate_datetime(date_debut, date_fin)


nombre de publications pour cette journée:  22
[1, 1, 0, 0, 0, 1, 0, 0, 1, 2, 0, 3, 3, 2, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1]
2023-01-01 00:00-00:59 1
Gouiaa Imen a créé une publication
2023-01-01 01:00-01:59 1
Masmoudi Hanen a créé une publication
2023-01-01 02:00-02:59 0
2023-01-01 03:00-03:59 0
2023-01-01 04:00-04:59 0
2023-01-01 05:00-05:59 1
Abid Fatma a créé une publication
2023-01-01 06:00-06:59 0
2023-01-01 07:00-07:59 0
2023-01-01 08:00-08:59 1
Ayedi Amin a créé une publication
2023-01-01 09:00-09:59 2
Chelly Kholdoun a créé une publication
Fakhfakh Najeh a créé une publication
2023-01-01 10:00-10:59 0
2023-01-01 11:00-11:59 3
Bouazize Yosra a créé une publication
Gouiaa Karim a créé une publication
Kharrat Ahmed a créé une publication
2023-01-01 12:00-12:59 3
Abid Lina a créé une publication
Ben Hmida Bacem a créé une publication
Bouazize Yosra a créé une publication
2023-01-01 13:00-13:59 2
Dridi Ameni a créé une publication
Bouraoui Yassine a créé une publication
2023-01-01 14:00

2023-01-07 06:00-06:59 1
Ben Hmida Yassmine a créé une publication
2023-01-07 07:00-07:59 0
2023-01-07 08:00-08:59 0
2023-01-07 09:00-09:59 2
Ben amar Bouthaina a créé une publication
Ben Ameur Hanen a créé une publication
2023-01-07 10:00-10:59 0
2023-01-07 11:00-11:59 3
Kharrat Ahmed a créé une publication
eleuch Assewer a créé une publication
Jabeur Amin a créé une publication
2023-01-07 12:00-12:59 2
Ben Hmida Bacem a créé une publication
Bouazize Yosra a créé une publication
2023-01-07 13:00-13:59 1
Bouraoui Yassine a créé une publication
2023-01-07 14:00-14:59 1
rebai Bacem a créé une publication
2023-01-07 15:00-15:59 1
Ben amar Bouthaina a créé une publication
2023-01-07 16:00-16:59 1
Dammak Imen a créé une publication
2023-01-07 17:00-17:59 0
2023-01-07 18:00-18:59 1
chebbi Nour a créé une publication
2023-01-07 19:00-19:59 1
Trabelsi Dani a créé une publication
2023-01-07 20:00-20:59 0
2023-01-07 21:00-21:59 1
Trabelsi Hamdi a créé une publication
2023-01-07 22:00-22:59 1
Fak

Ben Hmida Imen a créé une publication
Sallemi Karim a créé une publication
2023-01-13 10:00-10:59 0
2023-01-13 11:00-11:59 3
Ben Hmida Imen a créé une publication
Chelly Ameni a créé une publication
2023-01-13 12:00-12:59 3
Dridi Mohamed a créé une publication
Samet Rahma a créé une publication
rebai Ameni a créé une publication
2023-01-13 13:00-13:59 2
Samet Heni a créé une publication
Latrach Omar a créé une publication
2023-01-13 14:00-14:59 2
Ben rejeb Heni a créé une publication
Chelly Dani a créé une publication
2023-01-13 15:00-15:59 2
Chelly Fedi a créé une publication
chebbi Yosra a créé une publication
2023-01-13 16:00-16:59 1
Samet Ameni a créé une publication
2023-01-13 17:00-17:59 0
2023-01-13 18:00-18:59 0
2023-01-13 19:00-19:59 1
Hajkacem Assad a créé une publication
2023-01-13 20:00-20:59 0
2023-01-13 21:00-21:59 1
Smaoui Mahdi a créé une publication
2023-01-13 22:00-22:59 0
2023-01-13 23:00-23:59 0
nombre de publications pour cette journée:  20
[1, 1, 0, 0, 0, 1, 1, 0,

Dridi Assad a créé une publication
2023-01-19 05:00-05:59 0
2023-01-19 06:00-06:59 0
2023-01-19 07:00-07:59 0
2023-01-19 08:00-08:59 0
2023-01-19 09:00-09:59 2
Ayedi Ahmed a créé une publication
Smaoui Mahdi a créé une publication
2023-01-19 10:00-10:59 0
2023-01-19 11:00-11:59 2
Dridi Ameni a créé une publication
Ben Ayed Dani a créé une publication
2023-01-19 12:00-12:59 2
Bouazize Yassine a créé une publication
Smaoui Omar a créé une publication
2023-01-19 13:00-13:59 2
Dammak Kholdoun a créé une publication
chebbi Nour a créé une publication
2023-01-19 14:00-14:59 2
Smaoui Mahdi a créé une publication
Chouaibi Heni a créé une publication
2023-01-19 15:00-15:59 2
Abdelhedi Yassine a créé une publication
Ben Hmida Yassmine a créé une publication
2023-01-19 16:00-16:59 2
Samet Rahma a créé une publication
Ben Hmida Yassmine a créé une publication
2023-01-19 17:00-17:59 0
2023-01-19 18:00-18:59 0
2023-01-19 19:00-19:59 1
Chouaibi Yassmine a créé une publication
2023-01-19 20:00-20:59 0

Chelly Ameni a créé une publication
nombre de publications pour cette journée:  19
[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 3, 2, 2, 1, 1, 2, 0, 1, 1, 0, 0, 1, 0]
2023-01-25 00:00-00:59 1
Smaoui Mahdi a créé une publication
2023-01-25 01:00-01:59 0
2023-01-25 02:00-02:59 0
2023-01-25 03:00-03:59 1
Fakhfakh Karim a créé une publication
2023-01-25 04:00-04:59 1
Ayedi Ahmed a créé une publication
2023-01-25 05:00-05:59 0
2023-01-25 06:00-06:59 1
Latrach Kholdoun a créé une publication
2023-01-25 07:00-07:59 0
2023-01-25 08:00-08:59 0
2023-01-25 09:00-09:59 1
rebai Ameni a créé une publication
2023-01-25 10:00-10:59 0
2023-01-25 11:00-11:59 3
Ben Hmida Imen a créé une publication
Samet Ameni a créé une publication
krichen Raya a créé une publication
2023-01-25 12:00-12:59 2
Hajkacem Ghassen a créé une publication
Chouaibi Ahmed a créé une publication
2023-01-25 13:00-13:59 2
Ayedi Ahmed a créé une publication
Chelly Dani a créé une publication
2023-01-25 14:00-14:59 1
Dridi Assad a créé une publi

Chelly Dani a créé une publication
2023-01-31 03:00-03:59 0
2023-01-31 04:00-04:59 1
Abid Fatma a créé une publication
2023-01-31 05:00-05:59 1
Latrach Omar a créé une publication
2023-01-31 06:00-06:59 0
2023-01-31 07:00-07:59 0
2023-01-31 08:00-08:59 0
2023-01-31 09:00-09:59 1
Menif Yassine a créé une publication
2023-01-31 10:00-10:59 0
2023-01-31 11:00-11:59 2
Gouiaa Karim a créé une publication
Fakhfakh Karim a créé une publication
2023-01-31 12:00-12:59 3
Gouiaa Majdi a créé une publication
Ben Hmida Hamdi a créé une publication
Dridi Assad a créé une publication
2023-01-31 13:00-13:59 2
Fakhfakh Karim a créé une publication
Samet Heni a créé une publication
2023-01-31 14:00-14:59 1
Smaoui Omar a créé une publication
2023-01-31 15:00-15:59 1
Chouaibi Kholdoun a créé une publication
2023-01-31 16:00-16:59 2
Jabeur Amin a créé une publication
Bouazize Yosra a créé une publication
2023-01-31 17:00-17:59 0
2023-01-31 18:00-18:59 1
Bouazize Mohamed a créé une publication
2023-01-31 19

2023-02-06 04:00-04:59 0
2023-02-06 05:00-05:59 0
2023-02-06 06:00-06:59 0
2023-02-06 07:00-07:59 0
2023-02-06 08:00-08:59 0
2023-02-06 09:00-09:59 2
Menif Dani a créé une publication
Abdelkafi Karim a créé une publication
2023-02-06 10:00-10:59 2
Jallouli Najeh a créé une publication
Smaoui Omar a créé une publication
2023-02-06 11:00-11:59 2
Gouiaa Majdi a créé une publication
rebai Bacem a créé une publication
2023-02-06 12:00-12:59 3
Hajkacem Assad a créé une publication
Bouazize Mohamed a créé une publication
Abdelkafi Hanen a créé une publication
2023-02-06 13:00-13:59 2
Fitouri Mahdi a créé une publication
Dridi Ameni a créé une publication
2023-02-06 14:00-14:59 2
Dammak Kholdoun a créé une publication
Smaoui Mahdi a créé une publication
2023-02-06 15:00-15:59 1
Ben Hmida Nour a créé une publication
2023-02-06 16:00-16:59 2
Hajkacem Assad a créé une publication
rebai Hanen a créé une publication
2023-02-06 17:00-17:59 0
2023-02-06 18:00-18:59 0
2023-02-06 19:00-19:59 0
2023-02-

Samet Rahma a créé une publication
2023-02-11 19:00-19:59 1
eleuch Assewer a créé une publication
2023-02-11 20:00-20:59 0
2023-02-11 21:00-21:59 0
2023-02-11 22:00-22:59 1
Fitouri Samy a créé une publication
2023-02-11 23:00-23:59 0
nombre de publications pour cette journée:  17
[0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 3, 2, 1, 2, 2, 1, 0, 1, 0, 0, 0, 0, 1]
2023-02-12 00:00-00:59 0
2023-02-12 01:00-01:59 0
2023-02-12 02:00-02:59 0
2023-02-12 03:00-03:59 0
2023-02-12 04:00-04:59 0
2023-02-12 05:00-05:59 1
Abdelkafi Hanen a créé une publication
2023-02-12 06:00-06:59 0
2023-02-12 07:00-07:59 0
2023-02-12 08:00-08:59 1
Ayedi Ahmed a créé une publication
2023-02-12 09:00-09:59 2
Abid Fatma a créé une publication
Kharrat Bechir a créé une publication
2023-02-12 10:00-10:59 0
2023-02-12 11:00-11:59 3
krichen Raya a créé une publication
Jabeur Amin a créé une publication
Ben Ayed Dani a créé une publication
2023-02-12 12:00-12:59 2
Chelly Fedi a créé une publication
Hajkacem Assad a créé une public

2023-02-17 17:00-17:59 1
Chouaibi Ahmed a créé une publication
2023-02-17 18:00-18:59 0
2023-02-17 19:00-19:59 0
2023-02-17 20:00-20:59 0
2023-02-17 21:00-21:59 0
2023-02-17 22:00-22:59 1
Lahmer Mahdi a créé une publication
2023-02-17 23:00-23:59 1
Mallouli Dani a créé une publication
nombre de publications pour cette journée:  17
[1, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1]
2023-02-18 00:00-00:59 1
Hajkacem Ghassen a créé une publication
2023-02-18 01:00-01:59 1
Chouaibi Yassmine a créé une publication
2023-02-18 02:00-02:59 0
2023-02-18 03:00-03:59 0
2023-02-18 04:00-04:59 1
Hajkacem Ghassen a créé une publication
2023-02-18 05:00-05:59 0
2023-02-18 06:00-06:59 0
2023-02-18 07:00-07:59 0
2023-02-18 08:00-08:59 1
Ben rejeb Heni a créé une publication
2023-02-18 09:00-09:59 2
Ben Hmida Bacem a créé une publication
Fitouri Samy a créé une publication
2023-02-18 10:00-10:59 0
2023-02-18 11:00-11:59 2
rebai Ameni a créé une publication
Kharrat Ahmed a créé une 

2023-02-23 13:00-13:59 1
Chouaibi Yassmine a créé une publication
2023-02-23 14:00-14:59 1
Kharrat Bechir a créé une publication
2023-02-23 15:00-15:59 1
rebai Ameni a créé une publication
2023-02-23 16:00-16:59 2
Bouraoui Yassine a créé une publication
Ben Ameur Heni a créé une publication
2023-02-23 17:00-17:59 0
2023-02-23 18:00-18:59 1
Latrach Omar a créé une publication
2023-02-23 19:00-19:59 0
2023-02-23 20:00-20:59 0
2023-02-23 21:00-21:59 1
Tkaya Amin a créé une publication
2023-02-23 22:00-22:59 1
Fitouri Lina a créé une publication
2023-02-23 23:00-23:59 0
nombre de publications pour cette journée:  19
[1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0, 3, 3, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0]
2023-02-24 00:00-00:59 1
Samet Ameni a créé une publication
2023-02-24 01:00-01:59 1
Samet Ameni a créé une publication
2023-02-24 02:00-02:59 1
Chelly Ameni a créé une publication
2023-02-24 03:00-03:59 1
Smaoui Roua a créé une publication
2023-02-24 04:00-04:59 0
2023-02-24 05:00-05:59 0
2023-02-24 06:00

# L'ajout d'amplitude et de la tendance 

In [5]:
 
import random
from py2neo import Graph

# Connexion à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Récupérer tous les collaborateurs de la base de données
query = """
MATCH (c:Collaborateur)
RETURN c
"""
result = graph.run(query)

# Parcourir chaque collaborateur et effectuer les mises à jour
for record in result:
    collaborateur = record['c']

    # Générer la tendance aléatoire entre 30 et 100
    tendance = random.randint(30, 100)

    # Calculer l'amplitude en fonction de la tendance
    if random.random() * 100 < tendance:
        amplitude = random.uniform(0, 0.05)  # Entre 0% et 5%
    else:
        amplitude = random.uniform(-0.05, 0)  # Entre -5% et 0%

    # Calculer la motivation courante en utilisant l'amplitude et nb_news
    if collaborateur['motivation'] is not None:
        motivation_courante = collaborateur['motivation'] * (1 + amplitude)
    else:
        # Si la motivation n'est pas définie, attribuer une valeur par défaut
        motivation_courante = 0

    # Mettre à jour les attributs dans la base de données
    collaborateur['tendance'] = tendance
    collaborateur['amplitude'] = amplitude
    collaborateur['motivationCourante'] = motivation_courante

    # Enregistrer les modifications dans la base de données
    graph.push(collaborateur)


# Update de la tendance et l'amplitude a chaque collaborateur

In [6]:
import json
from neo4j import GraphDatabase

# Lecture du fichier JSON de configuration
with open("C:/Users/ESPRI/Desktop/PFE/Neo4j/pourcentage_amplitude_tendance.json", "r") as file:
    config = json.load(file)
pourcentage = config["pourcentage"] / 100  # Convertir en décimal



# Requête pour récupérer un collaborateur au hasard avec ses attributs de tendance et d'amplitude
query = """
    MATCH (c:Collaborateur)
    WITH c, RAND() AS rand
    RETURN c, rand
    ORDER BY rand
    LIMIT 1
"""
with driver.session() as session:
    result = session.run(query)
    record = result.single()
    collaborateur = record["c"]
    tendance_actuelle = collaborateur["tendance"]
    amplitude_actuelle = collaborateur["amplitude"]

    # Calcul des nouvelles valeurs de tendance et d'amplitude avec le pourcentage
    nouvelle_tendance = tendance_actuelle + (tendance_actuelle * pourcentage)
    nouvelle_amplitude = amplitude_actuelle + (amplitude_actuelle * pourcentage)

    # Requête de mise à jour pour mettre à jour les attributs de tendance et d'amplitude
    update_query = """
        MATCH (c:Collaborateur)
        WHERE ID(c) = $collaborateur_id
        SET c.tendance = $nouvelle_tendance, c.amplitude = $nouvelle_amplitude
    """
    session.run(update_query, collaborateur_id=collaborateur.id, nouvelle_tendance=nouvelle_tendance, nouvelle_amplitude=nouvelle_amplitude)

    # 
    print(f"Les attributs de tendance et d'amplitude du collaborateur {collaborateur.id} ont été mis à jour avec une augmentation de {config['pourcentage']}%.")


Les attributs de tendance et d'amplitude du collaborateur 176 ont été mis à jour avec une augmentation de 2%.


C:\Users\ESPRI\AppData\Local\Temp/ipykernel_18264/258119947.py:36: DeprecationWarning: `id` is deprecated, use `element_id` instead
  session.run(update_query, collaborateur_id=collaborateur.id, nouvelle_tendance=nouvelle_tendance, nouvelle_amplitude=nouvelle_amplitude)
C:\Users\ESPRI\AppData\Local\Temp/ipykernel_18264/258119947.py:39: DeprecationWarning: `id` is deprecated, use `element_id` instead
  print(f"Les attributs de tendance et d'amplitude du collaborateur {collaborateur.id} ont été mis à jour avec une augmentation de {config['pourcentage']}%.")


# Ajouter point pour chaque collaborateur les points

In [7]:
from py2neo import Graph
import schedule
import time
import threading

# Connexion à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Assurez-vous que la propriété nombre_de_points est définie pour le noeud Collaborateur
query = """
MATCH (c:Collaborateur)
SET c.nombre_de_points = 100
"""

graph.run(query)

# Fonction qui réinitialise les points de chaque collaborateur à 100
def reset_points():
    query = """
    MATCH (c:Collaborateur)
    SET c.nombre_de_points = 100
    """

    graph.run(query)

# Programmation de l'exécution de la fonction reset_points toutes les semaines
schedule.every().monday.at("00:00").do(reset_points)

# Fonction pour gérer la planification dans un thread séparé
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)

# Démarrage du thread de planification
schedule_thread = threading.Thread(target=run_schedule)
schedule_thread.start()


# Ajout les valeurs d'entreprise dans la base de données Neo4j

In [8]:
import json
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

def ajouter_valeurs_entreprise(tx, entreprise, valeurs):
    # Créer les nœuds pour les valeurs de l'entreprise
    for valeur in valeurs:
        query = """
        MERGE (v:Valeur {nom: $nom, points: $points})
        """
        tx.run(query, nom=valeur["nom"], points=valeur["points"])

        # Créer la relation "IS_RELATED_TO" entre l'entreprise et la valeur
        query = """
        MATCH (entreprise:Entreprise {nom: $entreprise_name})
        MATCH (v:Valeur {nom: $nom})
        MERGE (entreprise)<-[:IS_RELATED_TO]-(v)
        """
        tx.run(query, entreprise_name=entreprise_name, nom=valeur["nom"])

with open("c:/Users/ESPRI/Desktop/PFE/Neo4j/valeur_entreprise.json") as fichier:
    valeurs_entreprise = json.load(fichier)

with driver.session() as session:
    session.execute_write(ajouter_valeurs_entreprise,  entreprise['nom'], valeurs_entreprise)


# Ajouter les gifts_Model

In [9]:
import json
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

def ajouter_Model_Gift(tx, entreprise, Model):
    # Créer les nœuds pour les valeurs de l'entreprise
    for gift_model in Model:
        query = """
        MERGE (gm:gift_model {nom: $nom, cost: $cost})
        """
        tx.run(query, nom=gift_model["nom"], cost=gift_model["cost"])

        # Créer la relation "IS_RELATED_TO" entre l'entreprise et la valeur
        query = """
        MATCH (entreprise:Entreprise {nom: $entreprise_name})
        MATCH (mg:gift_model {nom: $nom})
        MERGE (entreprise)<-[:IS_RELATED_TO]-(mg)
        """
        tx.run(query, entreprise_name=entreprise_name, nom=gift_model["nom"])

with open("c:/Users/ESPRI/Desktop/PFE/Neo4j/Gift.json") as fichier:
    Gift = json.load(fichier)

with driver.session() as session:
    session.execute_write(ajouter_Model_Gift,  entreprise['nom'], Gift)


# Ajouter pour chaque collaborateur la % de Gift


In [10]:
from neo4j import GraphDatabase
import random


driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

def add_percentage_gift():
    with driver.session() as session:
        # Récupérer tous les collaborateurs ayant créé des publications
        query = """
        MATCH (c:Collaborateur)<-[:IS_CREATED_BY]-(p:Publication )
        RETURN c
        """
        result = session.run(query)
        collaborators_with_publications = [record["c"] for record in result]

        # Pour chaque collaborateur ayant créé des publications, attribuer un pourcentage de cadeaux aléatoire entre 20% et 80%
        for collaborator in collaborators_with_publications:
            percentage_gift = random.randint(20, 80)
            
            # Convertir le pourcentage en décimal (valeur entre 0 et 1)
            percentage_gift_decimal = percentage_gift 
            
            # Ajouter l'attribut "percentage_gift" au collaborateur dans la base de données
            query = f"""
            MATCH (c:Collaborateur {{nom: '{collaborator['nom']}', prenom: '{collaborator['prenom']}'}})
            SET c.percentage_gift = {percentage_gift_decimal}
            """
            session.run(query)

add_percentage_gift()

# %Gift_pourcentage_team et L'ajout de Gift

In [11]:
import random
from py2neo import Graph, Node, Relationship

uri = "bolt://localhost:7687"
graph = Graph(uri, auth=("neo4j", "nadou24474979"))

def get_model_gifts():
    query = """
    MATCH (mg:gift_model)
    RETURN mg
    """
    result = graph.run(query)
    return [record["mg"] for record in result]

def send_gifts():
    # Récupérer tous les model_gifts disponibles
    all_model_gifts = get_model_gifts()

    if not all_model_gifts:
        print("Aucun modèle de cadeau trouvé dans la base de données.")
        return

    # Récupérer tous les collaborateurs ayant l'attribut percentage_gift défini
    query = """
    MATCH (c:Collaborateur)
    WHERE c.percentage_gift IS NOT NULL
    RETURN c
    """
    result = graph.run(query)
    all_collaborators = [record["c"] for record in result]

    if not all_collaborators:
        print("Aucun collaborateur avec l'attribut 'percentage_gift' défini.")
        return

    # Parcourir chaque collaborateur et déterminer s'il envoie un cadeau
    for collaborator in all_collaborators:
        chance = random.randint(1, 100)
        team_percentage_gift = random.randint(10, 100)
        team_gift_chance = random.randint(1, 100)

        if chance <= collaborator["percentage_gift"]:
            # Récupérer le nom de l'équipe du collaborateur
            query = """
            MATCH (c:Collaborateur {nom: $nom, prenom: $prenom})<-[:HAS_EMPLOYEE]-(s:Structure { type: 'equipe'})
            RETURN s.nom AS equipe_nom
            """
            parameters = {"nom": collaborator["nom"], "prenom": collaborator["prenom"]}
            equipe_nom = graph.run(query, parameters=parameters).evaluate()

            if not equipe_nom:
                print(f"Le collaborateur {collaborator['nom']} {collaborator['prenom']} n'est associé à aucune équipe.")
                continue  # Passer au collaborateur suivant

            # Vérifier si le cadeau est pour un membre de la même équipe
            if team_gift_chance <= team_percentage_gift:
                query = """
                MATCH (c:Collaborateur)<-[:HAS_EMPLOYEE]-(:Structure {nom: $nom})
                RETURN c
                """
                recipient_collaborator = graph.run(query, parameters={"nom": equipe_nom}).evaluate()
            else:
                # Le cadeau est pour un membre d'une autre équipe
                query = """
                MATCH (c:Collaborateur)
                RETURN c
                """
                recipient_collaborator = graph.run(query).evaluate()

            # Choisir un model_gift aléatoirement
            model_gift = random.choice(all_model_gifts)

            # Vérifier si le collaborateur a assez de points pour envoyer le cadeau
            if collaborator["nombre_de_points"] < model_gift["cost"]:
                print(f"Le collaborateur {collaborator['nom']} {collaborator['prenom']} n'a pas assez de points pour envoyer le cadeau.")
                continue  # Passer au collaborateur suivant sans envoyer de cadeau

            # Créer le nœud cadeau et la relation entre le donneur, le cadeau, le model_gift et le receveur
            cadeau = Node("Cadeau", recipient_nom=recipient_collaborator["nom"], recipient_prenom=recipient_collaborator["prenom"])
            giver_rel = Relationship(collaborator, "GIVES", cadeau)
            to_rel = Relationship(cadeau, "TO", recipient_collaborator)
            based_on_rel = Relationship(cadeau, "BASED_ON", model_gift)

            graph.create(cadeau)
            graph.create(giver_rel)
            graph.create(to_rel)
            graph.create(based_on_rel)

            # Soustraire le coût du model_gift du nombre de points du collaborateur
            new_percentage_gift = collaborator["nombre_de_points"] - model_gift["cost"]
            query = f"""
            MATCH (c:Collaborateur {{nom: '{collaborator['nom']}', prenom: '{collaborator['prenom']}'}})
            SET c.nombre_de_points = {new_percentage_gift}
            """
            graph.run(query)

send_gifts()


Le collaborateur Heni Ben Ameur n'est associé à aucune équipe.
Le collaborateur Majdi Gouiaa n'est associé à aucune équipe.
Le collaborateur Assad Hajkacem n'est associé à aucune équipe.
Le collaborateur Ameni Chelly n'est associé à aucune équipe.
Le collaborateur Assewer eleuch n'est associé à aucune équipe.
Le collaborateur Kholdoun Chouaibi n'est associé à aucune équipe.
Le collaborateur Assewer eleuch n'est associé à aucune équipe.
Le collaborateur Raya krichen n'est associé à aucune équipe.
Le collaborateur Ameni Samet n'est associé à aucune équipe.
Le collaborateur Yosra Bouazize n'est associé à aucune équipe.


# Lier les news avec les valeurs de l'entreprise

In [12]:
from neo4j import GraphDatabase
import random

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

def relier_publications_avec_valeurs_aleatoires(tx):
    publications_result = tx.run("MATCH (p:Publication) RETURN p")

    for record in publications_result:
        publication = record['p'] 

        
        nombre_valeurs_par_publication = random.randint(1, 4)

        
        query = """
        MATCH (p:Publication) WHERE id(p) = $publication_id
        WITH p
        MATCH (v:Valeur)
        WITH p, collect(v) as valeurs
        UNWIND valeurs[0..$nombre_valeurs_par_publication] as valeur
        MERGE (p)-[:IS_RELATED_TO]->(valeur)
        """
        tx.run(query, publication_id=publication.id, nombre_valeurs_par_publication=nombre_valeurs_par_publication)

with driver.session() as session:
    session.execute_write(relier_publications_avec_valeurs_aleatoires)


C:\Users\ESPRI\AppData\Local\Temp/ipykernel_18264/2179360023.py:25: DeprecationWarning: `id` is deprecated, use `element_id` instead
  tx.run(query, publication_id=publication.id, nombre_valeurs_par_publication=nombre_valeurs_par_publication)
